In [1]:
import sys
sys.path.append("..")
from config import aws_key, aws_secret
from config import mongo_username, mongo_password, mongo_server_name
from config import postgres_server_name, postgres_database, postgres_username, postgres_port, postgres_password

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session

In [3]:
import pandas as pd

In [4]:
db_url = f"postgres://{postgres_username}:{postgres_password}@{postgres_server_name}:{postgres_port}/{postgres_database}"
engine = create_engine(db_url)
conn = engine.connect()
session = Session(bind=engine)

# Crime

In [8]:
query = session.query(' * from all_crime')
crime_df = pd.read_sql(query.statement, engine)


In [9]:
crime_df['report_date'] = pd.to_datetime(crime_df['report_date'])
crime_df.head()

,Index,crime_type,report_date,Beat,Neigborhood,NPU,lat,lon
0,0,LARCENY-NON VEHICLE,2009-02-28,301.0,Adair Park,V,33.72710,-84.41360
1,1,LARCENY-NON VEHICLE,2009-02-28,509.0,Downtown,M,33.76158,-84.38496
2,2,LARCENY-NON VEHICLE,2009-02-28,210.0,Lenox,B,33.84676,-84.36212
3,3,LARCENY-NON VEHICLE,2009-02-28,204.0,Berkeley Park,D,33.80448,-84.41348
4,4,LARCENY-NON VEHICLE,2009-02-28,609.0,Edgewood,O,33.75674,-84.34619


In [ ]:
def crime_variable (row):
    if row['crime_type'] == 'HOMICIDE':
        return 1
    if row['crime_type'] == 'ROBBERY-PEDESTRIAN':
        return 1
    if row['crime_type'] == 'BURGLARY-RESIDENCE':
        return 1
    if row['crime_type'] == 'AGG ASSAULT':
        return 1
    return 0

# applying the function to go through every row and create a new column called 'rainy'

crime_df['person_crime'] = crime_df.apply(lambda row:crime_variable(row),axis = 1)

In [ ]:
crime_df['year'] = pd.DatetimeIndex(crime_df['report_date']).year
crime_df['month'] = pd.DatetimeIndex(crime_df['report_date']).month

In [ ]:
crime_person_df = crime_df[["person_crime","year", "month"]]

In [ ]:
crime_person_time_df = crime_person_df.groupby(["year", "month"]).sum().reset_index()
crime_person_time_df.head()

# Weather

In [10]:
query = session.query(' * from hist_weather')
atl_weather_df = pd.read_sql(query.statement, engine)


In [11]:
atl_weather_df['date_simple'] = pd.to_datetime(atl_weather_df['date_simple'])
atl_weather_df.head()

,id,date,date_iso,weather_id,weather_group,weather,weather_icon,temperature,temp_min,temp_max,feels_like,pressure,humidity,wind_speed,cloud_percentage,date_simple
0,0,1230768000,2009-01-01 00:00:00 +0000 UTC,800,Clear,sky is clear,01n,39.78,39.20,41.12,25.48,1023,40,16.11,1,2009-01-01
1,1,1230771600,2009-01-01 01:00:00 +0000 UTC,800,Clear,sky is clear,01n,39.02,37.40,41.49,30.45,1024,40,5.82,1,2009-01-01
2,2,1230775200,2009-01-01 02:00:00 +0000 UTC,800,Clear,sky is clear,01n,37.81,37.04,40.35,28.63,1025,42,6.93,1,2009-01-01
3,3,1230778800,2009-01-01 03:00:00 +0000 UTC,800,Clear,sky is clear,01n,35.82,35.55,35.96,27.25,1025,45,5.82,1,2009-01-01
4,4,1230782400,2009-01-01 04:00:00 +0000 UTC,800,Clear,sky is clear,01n,34.59,33.80,35.52,27.39,1025,47,3.36,1,2009-01-01


In [ ]:
atl_weather_df['year'] = pd.DatetimeIndex(atl_weather_df['date_simple']).year
atl_weather_df['month'] = pd.DatetimeIndex(atl_weather_df['date_simple']).month

In [ ]:
real_feel_Time_df = atl_weather_df[["feels_like","year", "month"]]
real_feel_TimeGroup_df = real_feel_Time_df.groupby(["year", "month"]).mean().reset_index()
real_feel_TimeGroup_df.head()

# Join Crime and Weather

In [ ]:
personCrime_temp_df = pd.merge(crime_person_time_df,real_feel_TimeGroup_df, how = 'inner', on = ['year','month'])

In [ ]:
personCrime_temp_df["day"] = 1

In [ ]:
personCrime_temp_df["date"] = pd.to_datetime(personCrime_temp_df[['year', 'month', 'day']])

In [ ]:
org_crime_temp_df = personCrime_temp_df[["date","person_crime","feels_like"]]
org_crime_temp_df = org_crime_temp_df.set_index("date")

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Date')
ax1.set_ylabel('Count Crimes', color=color)
ax1.plot(org_crime_temp_df.index, org_crime_temp_df.person_crime, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Real Feel', color=color)  # we already handled the x-label with ax1
ax2.plot(org_crime_temp_df.index, org_crime_temp_df.feels_like, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()